In [ ]:
import numpy as np
import pandas as pd
from scipy.io import *
from pylab import *
from pandas import DataFrame as df
import scipy
from PIL import Image
# import random
from scipy import stats
from pyclustering.cluster.kmeans import kmeans
from pyclustering.utils.metric import type_metric, distance_metric
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from scipy.spatial import distance

In [ ]:
FPPs = loadmat('Matrix120.mat')['s'][0]
communitiest = loadmat('Cluster120.mat')['b'][0]


In [ ]:
# for i in range(len(FPP)):
#     FPP_= FPP[i].reshape(len(FPP[i]), 51, 20)
#     for j in range(len(FPP_)):
#         title('Rat %i FPP %i'%(i+1,j+1 ))
#         imshow(FPP_[j],extent=[-np.pi, np.pi, 20, 120],aspect='auto' )
#         ylabel('Frequencies(Hz)')
#         xlabel('Phase')
#         colorbar()
# #         savefig('Rat%i_FPP_%i'%(i+1, j+1))
#         show()
np.array([1,6, 4, 5]).shape

In [ ]:
def pearson_dist(x, y):
    r = stats.pearsonr(x, y)[0]
    return (1 - r) / 2

def kmeans_(FPP, k=3):
    # define a custom metric
    metric = distance_metric(type_metric.USER_DEFINED, func=pearson_dist)
    # carry out a km++ init
    initial_centers = kmeans_plusplus_initializer(FPP, k).initialize()
    # execute kmeans
    kmeans_instance = kmeans(FPP, initial_centers, metric=metric)
    # run cluster analysis
    kmeans_instance.process()
    # get clusts
    clusters = kmeans_instance.get_clusters()

    return clusters
                    

In [ ]:
#  Get amount of clusters expected through community clustering
maxima=[]
for i in range(1,16, 2):
    maxima.append(np.mean(communitiest[i]))
PFC_k= round(np.mean(np.array(maxima)))
maxima2=[]
for i in range(0,16,2):
    maxima2.append(np.mean(communitiest[i]))
HPC_k=round(np.mean(np.array(maxima2)))


In [ ]:
# Retrieve HMM clustering labels

pfc_clusters=[]
for i in range(1,16,2):
    rat = FPPs[i]
    c =PFC_k
    pfc_clusters.append(kmeans_(rat, k=c))
hpc_clusters=[]
for i in range(0,16,2):
    rat = FPPs[i]
    c =HPC_k
    hpc_clusters.append(kmeans_(rat, k=c))


In [ ]:
def recluster(FPP, clusters, start=0, step=2):
    '''
    Input: all FPPs, cluster labels from kmeans, start and step point rat iteration
    Export: all clusters for all rats
    Retrieve the data within the clusters
    '''
    pls = []
    for i in range(start,16,step):
        cl_ = []
        for j in range(0,len(np.array(clusters[i//2]))):
            p=i
            cls_=[]
            F= FPP[i]
            cluster=F[np.array(clusters[i//2])[j]]
#             print(shape(cl_))
            cl_.append(cluster)
        pls.append(cl_)
    return pls

In [ ]:
pls_pfc = recluster(FPPs, pfc_clusters, 1)
pls_hpc = recluster(FPPs, hpc_clusters)

In [ ]:
def pearsoncor(x, y):
    r = stats.pearsonr(x, y)[0]
    return r
def mahalanobis_dist(x, y, lr=False):
    m =list(zip(x, y))
    try:
        iv = np.linalg.inv(np.cov(m))
    except:
        iv = np.linalg.pinv(np.cov(m)) 
    return distance.mahalanobis(x, y,iv)
def lrratio(x):
    l=(stats.norm.cdf(np.array(x)**2))
    return (1-l)/len(x)
def intracor(clusters, FPP_k, pearsoncor=False, mahalanobis=False):
    '''
    Retrieve correlations between the points in a specific cluster
    Input: Singular FPP, mean of all FPP within cluster, distance function indicator
    '''
    intra = []
    for j in range(len(clusters)):
        FPP_i=clusters[j]
        if mahalanobis:
            intra.append(mahalanobis_dist(FPP_i, FPP_k))
        elif pearsoncor:
            intra.append(pearsoncor(FPP_i, FPP_k))
        #Default pearson dist
        else:
            intra.append(pearson_dist(FPP_i, FPP_k))
    return intra

def extracor(clusters, FPP_k, pearsoncor=False,lr=False, mahalanobis=False):
    '''
    Retrieve correlations between the points in a specific cluster and the other clusters
    Input: Singular FPP, mean of all FPP outside of clusters, distance function indicator
    '''
    extra=[]
    FPP_k= np.mean(FPP_k, axis=0)
    for j in range(len(clusters)):
        FPP_i=clusters[j]
        if mahalanobis or lr:
            extra.append(mahalanobis_dist(FPP_i, FPP_k))
        elif pearsoncor:
            extra.append(pearsoncor(FPP_i, FPP_k))
        #Default pearson dist
        else:
            extra.append(pearson_dist(FPP_i, FPP_k))
    if lr:
        return np.nanmean(lrratio(extra))
    return np.nanmean(extra)

In [ ]:
# def intraextra(pls):
#     cluster_in=[]
#     cluster_ex=[]
#     for i in range(8):

#         f= pls[i].copy()
#         extra=[]
#         intra=[]
#         for c in range(len(pls[i])):
#             cluster = np.array(pls[i][c]).reshape(len(pls[i][c]), 51, 20)
#             cluster= np.mean(cluster, axis=0)
#             f.pop(c)
#             FPP_k_mean = f.copy()
#             FPP_k = np.mean(cluster, axis=1)
#             intr=[]
#             extr = []
# #             print(shape(FPP_k), shape(f))
#             for j in range(20):
#                 FPP_i=cluster[:,j]
#                 intr.append(intracor(FPP_i, FPP_k))
#                 extr.append(extracor(FPP_i, FPP_k_mean[0]))
#             extra.append(max(extr))
#             intra.append(np.mean(intr))
#             f=pls[i].copy()
#         cluster_in.append(intra)
#         cluster_ex.append(extra)
#     return cluster_in, cluster_ex

In [ ]:
def gamma_sort(pls, region, threshold = 0.95):
    '''
    Input: clusters for all rats, region and threshold for centroid of gamma
    Output:mean (res/results) of all FPP in clusters for all rats, significancies for all distance measures  
    Sort into gamma clusters, retrieve significance and plot
    note: when num>1 iterate over ex_clusters
    '''
    res=[]
    accuracy_ex=[]
    accuracy_in=[]
    M_accuracy_ex=[]
    M_accuracy_in=[]
#     lr_accuracy =[]
    for j in range(len(pls)):
        cls_=np.array(pls[j])
        ex_clusters= list(cls_.copy())
        clusters = []
        in_acc=[]
        ex_acc=[]
#         lr_acc=[]
        M_ex_clusters= list(cls_.copy())
        M_in_acc=[]
        M_ex_acc=[]
#         lr_ex_clusters= list(cls_.copy())
        for i in range(len(cls_)):
            cluster_init=np.array(cls_[i])
#             gamma_peak = threshold*max(np.array([np.max(c) for c in cluster_init]))
            cluster=cluster_init.reshape(len(cluster_init),51,20)
            masked_cluster =np.mean(cluster, axis=0)
            #Measure significance
            
            ex_clusters.pop(i)
            for c in range(len(ex_clusters)):
                #Pearson
                intra = np.nanmean(intracor(cluster_init, np.mean(cluster_init, axis=0)))
                extra=extracor(cluster_init, np.array(ex_clusters[c]))
                in_acc.append(intra)
                ex_acc.append(extra)
                ex_clusters=list(cls_.copy())
                #mahalanobis
                M_ex_clusters.pop(i)
                M_intra = np.nanmean(intracor(cluster_init, np.mean(cluster_init, axis=0), mahalanobis=True))
                M_extra=extracor(cluster_init, np.array(M_ex_clusters[c]), mahalanobis=True)
                M_in_acc.append(M_intra)
                M_ex_acc.append(M_extra)
                M_ex_clusters=list(cls_.copy())
                #Lratio
#                 lr_ex_clusters.pop(i)
#                 lr=extracor(cluster_init, np.array(lr_ex_clusters[c]),lr=True)
#                 lr_acc.append(lr)
#                 lr_ex_clusters=list(cls_.copy())

            imshow(masked_cluster,extent=[-np.pi, np.pi, 120, 20],aspect='auto', cmap = 'hot' )
            ylabel('Frequencies(Hz)')    
            xlabel('Phase')
            colorbar()
            if j>=4:
                title('Rat %i cluster %i %s'%(j+2,i+1, region))

                savefig('../Clusters/HomecageKmeansMeanClusters/Rat %i cluster %i %s'%(j+2,i+1, region))
            else:
                title('Rat %i cluster %i %s'%(j+1,i+1, region))
                savefig('../Clusters/HomecageKmeansMeanClusters/Rat %i cluster %i %s'%(j+1,i+1, region))
            
            show()
            #show average bin
#             for c in range(20):
#                 imshow(np.array([av_cluster[:,c]]).T,aspect='auto' )
#                 ylabel('Frequencies(Hz)')
#                 colorbar()
#                 if j>=4:
#                     title('Rat %i cluster %i cycle %i'%(j+2,i+1, c+1 ))

#     #                 savefig('Rat %i cluster %i %s cycle %i'%(j+2,i+1, region, c+1))
#                 else:
#                     title('Rat %i cluster %i cycle %i'%(j+1,i+1, c+1))
#     #                 savefig('Rat %i cluster %i %s cycle %i'%(j+1,i+1, region, c+1)
#                 show()
#             l=random.choices(range(len(cluster)), k=3)
#             for f in l:
#                 imshow(cluster[f],extent=[-np.pi, np.pi, 120, 20],aspect='auto', cmap = 'hot')
#                 ylabel('Frequencies(Hz)')
#                 colorbar()
#                 xlabel('Phase')
#                 if j>=4:
#                     title('Rat %i cluster %i random sample %s'%(j+2,i+1, region))

# #                     savefig('Rat %i cluster %i random sample %i'%(j+2,i+1, f+1))
#                 else:
#                     title('Rat %i cluster %i random sample %s'%(j+1,i+1, region))
# #                     savefig('Rat %i cluster %i random sample %i'%(j+1,i+1, f+1))
#                 show()
            
            clusters.append(masked_cluster)
        accuracy_ex.append(ex_acc)
        accuracy_in.append(in_acc)
        M_accuracy_ex.append(M_ex_acc)
        M_accuracy_in.append(M_in_acc)
#         lr_accuracy.append(lr_acc)
        res.append(clusters)
    return res, accuracy_ex, accuracy_in, M_accuracy_ex,M_accuracy_in

In [ ]:
gamma_clusters_pfc,pfc_cluster_ex, pfc_cluster_in, mpfc_cluster_ex, mpfc_cluster_in=gamma_sort(pls_pfc,  'PFC')
gamma_clusters_hpc,hpc_cluster_ex,hpc_cluster_in, mhpc_cluster_ex, mhpc_cluster_in=gamma_sort(pls_hpc,  'HPC')

In [ ]:
df_clusters =  df(columns =['Rat', 'Cluster_Number','Mutation/Control', 'Condition','PFC_Clusters', 'HPC_Clusters', 'PFC_Gamma_Clusters', 'HPC_Gamma_Clusters'])
df_clusters['PFC_Clusters']= df_clusters['PFC_Clusters'].astype(object)
df_clusters['HPC_Clusters']= df_clusters['HPC_Clusters'].astype(object)
df_clusters['PFC_Gamma_Clusters']= df_clusters['PFC_Gamma_Clusters'].astype(object)
df_clusters['HPC_Gamma_Clusters']= df_clusters['HPC_Gamma_Clusters'].astype(object)
mclist=['Control', 'Control','Mutation', 'Mutation',  'Control', 'Mutation', 'Mutation','Control']
for i in range(0,8):
    j=i+1
    if j >= 5:
        j+=1
    for num in range(len(gamma_clusters_hpc[i])):
        hpc_extra = hpc_cluster_ex[i][num]
        hpc_intra =hpc_cluster_in[i][num]
        pfc_extra =pfc_cluster_ex[i][num]
        pfc_intra = pfc_cluster_in[i][num]
        m_hpc_extra = mhpc_cluster_ex[i][num]
        m_hpc_intra =mhpc_cluster_in[i][num]
        m_pfc_extra =mpfc_cluster_ex[i][num]
        m_pfc_intra = mpfc_cluster_in[i][num]
#         lr_hpc_extra = lr_hpc[i][num]
#         lr_pfc_extra =lr_pfc[i][num]
        df_clusters=df_clusters.append({'Rat':j,'Cluster_Number':num+1,'Mutation/Control': mclist[i],'Condition':'HomeCage', 'PearsonDistance_PFC_Intra_Cluster': pfc_intra,\
                                        'PearsonDistance_PFC_Inter_cluster':pfc_extra,'PearsonDistance_HPC_Intra_Cluster': hpc_intra, \
                                        'PearsonDistance_HPC_Inter_cluster':hpc_extra,'MahalanobisDistance_PFC_Intra_Cluster': m_pfc_intra,\
                                        'MahalanobisDistance_PFC_Inter_cluster':m_pfc_extra,'MahalanobisDistance_HPC_Intra_Cluster': m_hpc_intra, \
                                        'MahalanobisDistance_HPC_Inter_cluster':m_hpc_extra,'PFC_Clusters':pfc_clusters[i][num], 'HPC_Clusters': hpc_clusters[i][num], \
                                        'PFC_Gamma_Clusters':gamma_clusters_pfc[i][num], 'HPC_Gamma_Clusters':gamma_clusters_hpc[i][num]}, ignore_index=True)

In [ ]:
df_clusters.to_csv('../Clusters/HomecageKmeansMeanClusters/Clustersk_120_2_homecage.csv')